In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas
import hvplot as hv
from bokeh.models import HoverTool


Read data

In [3]:
ridership = pd.read_csv('data/ridership_detail.csv')

/var/folders/1w/nb8zzzyd2b19zfmmnsf651j00000gn/T/ipykernel_99345/3374443073.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  ridership = pd.read_csv('data/ridership_detail.csv')


In [4]:
ridership['total'] = ridership['ridership'] + ridership['transfers']
ridership_station_hour = ridership.groupby(['station_complex_id', 'hour'])['total'].mean().reset_index()
ridership_station_hour.head()

,station_complex_id,hour,total
0,A002,0,265.441494
1,A002,1,92.681063
2,A002,2,43.101836
3,A002,3,29.703086
4,A002,4,39.319767


In [5]:
stations = pd.read_csv('data/stations.csv')

ridership_station_hour = ridership_station_hour.merge(stations, how='left', on='station_complex_id')

station_clsutering = ridership_station_hour.copy()

# drop station_complex_id, station_complex, Georeference, total_log
station_clsutering = station_clsutering[['station_complex_id', 'hour', 'total']]
station_clsutering = station_clsutering.reset_index(drop=True)
station_clsutering.head()


,station_complex_id,hour,total
0,A002,0,265.441494
1,A002,1,92.681063
2,A002,2,43.101836
3,A002,3,29.703086
4,A002,4,39.319767


In [6]:
# pivot table, set column name format as hour_1, hour_2, hour_3, ...
station_clsutering_pivot = station_clsutering.pivot_table(index='station_complex_id', columns='hour', values='total')
station_clsutering_pivot.columns = ['hour_' + str(col) for col in station_clsutering_pivot.columns]
station_clsutering_pivot.reset_index(inplace=True)
station_clsutering_pivot.head()

,station_complex_id,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,A002,265.441494,92.681063,43.101836,29.703086,39.319767,88.238372,259.573920,585.364618,914.568106,...,1679.943522,2236.216777,2807.029070,3506.975083,2679.514950,1684.965947,1152.619285,745.310889,651.492512,498.916736
1,A006,58.932803,16.265928,7.508180,4.381881,4.423404,9.220887,30.985605,61.944962,93.859322,...,501.001686,721.064081,911.569386,1097.922494,866.514743,511.009251,321.417157,223.557530,232.862944,151.206081
2,A010,222.156846,75.742525,37.545000,21.404007,19.550459,34.461794,121.519934,305.490033,518.931894,...,904.612957,1230.863787,1527.372924,1918.554817,1600.022425,1053.206983,772.190357,717.019950,781.135607,505.402998
3,A013,220.243154,82.301495,37.846667,22.427736,17.651591,19.599668,54.874585,140.576412,307.607143,...,536.240033,701.229426,971.636741,1396.064838,1093.290939,712.262677,576.815308,710.218802,709.750208,453.258333
4,A022,691.831535,293.986711,150.985833,84.716777,87.146179,195.255814,716.936877,1494.406146,2257.887043,...,2702.924419,3315.724252,4174.717608,5801.048173,4952.907807,3368.647010,2667.751455,2150.704073,1766.876872,1290.083126


In [7]:
# add latitude and longitude
station_clsutering_all = station_clsutering_pivot.merge(stations[['station_complex_id', 'latitude', 'longitude']], how='left', on='station_complex_id')

# drop station_complex_id
station_clsutering_training = station_clsutering_all.drop('station_complex_id', axis=1)

station_clsutering_training.head()

,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,...,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,latitude,longitude
0,265.441494,92.681063,43.101836,29.703086,39.319767,88.238372,259.573920,585.364618,914.568106,825.715947,...,2807.029070,3506.975083,2679.514950,1684.965947,1152.619285,745.310889,651.492512,498.916736,40.762661,-73.967255
1,58.932803,16.265928,7.508180,4.381881,4.423404,9.220887,30.985605,61.944962,93.859322,117.845439,...,911.569386,1097.922494,866.514743,511.009251,321.417157,223.557530,232.862944,151.206081,40.764812,-73.973351
2,222.156846,75.742525,37.545000,21.404007,19.550459,34.461794,121.519934,305.490033,518.931894,526.752492,...,1527.372924,1918.554817,1600.022425,1053.206983,772.190357,717.019950,781.135607,505.402998,40.764664,-73.980659
3,220.243154,82.301495,37.846667,22.427736,17.651591,19.599668,54.874585,140.576412,307.607143,377.771595,...,971.636741,1396.064838,1093.290939,712.262677,576.815308,710.218802,709.750208,453.258333,40.759899,-73.984138
4,691.831535,293.986711,150.985833,84.716777,87.146179,195.255814,716.936877,1494.406146,2257.887043,1937.913621,...,4174.717608,5801.048173,4952.907807,3368.647010,2667.751455,2150.704073,1766.876872,1290.083126,40.749718,-73.987823


In [8]:
# normalize the data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

station_clsutering_all_scaled = scaler.fit_transform(station_clsutering_training)

# change the scale of latitude and longitude to give them more weight

station_clsutering_all_scaled[:, -2] = station_clsutering_all_scaled[:, -2] * 5
station_clsutering_all_scaled[:, -1] = station_clsutering_all_scaled[:, -1] * 5

# Spectral Clustering

from sklearn.cluster import SpectralClustering

spectral_model = SpectralClustering(n_clusters=15, random_state=0, affinity='nearest_neighbors')

spectral_model.fit(station_clsutering_all_scaled)

station_clsutering_all_prediction = station_clsutering_all.copy()
station_clsutering_all_prediction['cluster'] = spectral_model.labels_
station_clsutering_all_prediction.head()

,station_complex_id,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,...,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,latitude,longitude,cluster
0,A002,265.441494,92.681063,43.101836,29.703086,39.319767,88.238372,259.573920,585.364618,914.568106,...,3506.975083,2679.514950,1684.965947,1152.619285,745.310889,651.492512,498.916736,40.762661,-73.967255,11
1,A006,58.932803,16.265928,7.508180,4.381881,4.423404,9.220887,30.985605,61.944962,93.859322,...,1097.922494,866.514743,511.009251,321.417157,223.557530,232.862944,151.206081,40.764812,-73.973351,14
2,A010,222.156846,75.742525,37.545000,21.404007,19.550459,34.461794,121.519934,305.490033,518.931894,...,1918.554817,1600.022425,1053.206983,772.190357,717.019950,781.135607,505.402998,40.764664,-73.980659,11
3,A013,220.243154,82.301495,37.846667,22.427736,17.651591,19.599668,54.874585,140.576412,307.607143,...,1396.064838,1093.290939,712.262677,576.815308,710.218802,709.750208,453.258333,40.759899,-73.984138,1
4,A022,691.831535,293.986711,150.985833,84.716777,87.146179,195.255814,716.936877,1494.406146,2257.887043,...,5801.048173,4952.907807,3368.647010,2667.751455,2150.704073,1766.876872,1290.083126,40.749718,-73.987823,11


In [9]:
station_cluster = station_clsutering_all_prediction[['station_complex_id', 'cluster']]
# merge with stations to get the station information
station_cluster_detail = station_cluster.merge(stations, how='left', on='station_complex_id')
station_cluster_detail.head()

,station_complex_id,cluster,station_complex,borough,routes,latitude,longitude,Georeference
0,A002,11,"Lexington Av (N,R,W)/59 St (4,5,6)",M,"5,N,R,4,,W,6",40.762661,-73.967255,POINT (-73.96725463867188 40.76266098022461)
1,A006,14,"5 Av-59 St (N,R,W)",M,"N,R,W",40.764812,-73.973351,POINT (-73.97335052490234 40.76481246948242)
2,A010,11,"57 St-7 Av (N,Q,R,W)",M,"N,R,Q,W",40.764664,-73.980659,POINT (-73.98065948486328 40.76466369628906)
3,A013,1,"49 St (N,R,W)",M,"N,R,W",40.759899,-73.984138,POINT (-73.98413848876953 40.7598991394043)
4,A022,11,"34 St-Herald Sq (B,D,F,M,N,Q,R,W)",M,"F,M,N,R,Q,B,W,D",40.749718,-73.987823,POINT (-73.98782348632812 40.749717712402344)


In [10]:
p_clsuter = station_cluster_detail.hvplot.points(
    'longitude', 'latitude', geo=True, tiles='CartoLight', color='cluster', 
    size=10, frame_width=700, frame_height=500, cmap='plasma', hover_cols=['station_complex'],
)

from holoviews import opts

hover_tool = HoverTool(tooltips=[
    ("Station", "@station_complex"),
    ("Cluster", "@cluster"),
    # Add any other columns you'd like to show on hover
])

p_clsuter = p_clsuter.opts(
    opts.Points(tools=[hover_tool])
)

hv.output(p_clsuter)


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (cluster,station_complex)

In [1]:
station_cluster.to_csv('data/stations_cluster.csv', index=False)

NameError: name 'station_cluster' is not defined